In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from codecarbon import EmissionsTracker
import os
import utils
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: '%.10f' % x)

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn


In [2]:
splits = utils.get_train_val_test_split()

X_train, y_train = splits['train']['X'], splits['train']['y']
X_val, y_val = splits['val']['X'], splits['val']['y']
X_test, y_test = splits['test']['X'], splits['test']['y']
y_train.dtype


dtype('float64')

In [3]:
from typing import Tuple


class SuicideDataSet(Dataset):
    def __init__(self, split: str = 'train'):
        self.split = split
        self.splits = utils.get_train_val_test_split()

    def __len__(self):
        return len(self.splits[self.split]['y'])

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        ftrs = self.splits[self.split]['X'][idx]
        label = self.splits[self.split]['y'][idx]
        ftrs = torch.tensor(ftrs).float()
        label = torch.tensor(label).float()
        return ftrs, label


In [4]:
class SuicideRegressor(nn.Module):
    def __init__(self, in_features) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64,8),
            nn.ReLU(),
            nn.Linear(8,1),
        )

    def forward(self, x):
        return self.model(x)

In [5]:
class SuicideRegressorBN(nn.Module):
    def __init__(self, in_features) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d(num_features=in_features),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=256),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=64),
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=8),
            nn.Linear(8, 1),
        )

    def forward(self, x):
        return self.model(x)

In [6]:
class SuicideRegressorDrop(nn.Module):
    def __init__(self, in_features) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d(num_features=in_features),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.BatchNorm1d(num_features=512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.BatchNorm1d(num_features=256),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.BatchNorm1d(num_features=64),
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.BatchNorm1d(num_features=8),
            nn.Linear(8, 1),
        )

    def forward(self, x):
        return self.model(x)

## Train model

In [7]:
train_ds = SuicideDataSet(split='train')
train_loader = DataLoader(dataset=train_ds, batch_size=1788)

val_ds = SuicideDataSet(split='val')
val_loader = DataLoader(dataset=val_ds, batch_size=276)

a = iter(train_loader)
b = a._next_data()
print(b)
# print(b)


[tensor([[ 0.0710, -0.2217,  1.0030,  ...,  0.6062,  0.1918,  1.3674],
        [ 0.3894,  0.9704, -0.9970,  ..., -1.3249,  2.2114,  1.7722],
        [-0.7782, -0.2217,  1.0030,  ..., -0.8897, -0.1976, -0.5043],
        ...,
        [-0.7782, -0.2217,  1.0030,  ..., -1.0257, -0.1255, -0.5228],
        [-0.3536, -0.2217, -0.9970,  ..., -0.4817, -0.3707, -0.2547],
        [-0.7782, -0.2217, -0.9970,  ..., -1.1073, -0.0678, -0.4936]]), tensor([12.0800,  3.6900, 44.8300,  ..., 70.7500, 16.3000,  9.8300])]


In [8]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
writer = SummaryWriter(log_dir=f"./tb_logs/{datetime.now().strftime('%Y-%M-%d_%H:%m')}")



In [9]:
model = SuicideRegressorDrop(in_features=9)

dataiter = iter(train_loader)
X, y = next(dataiter)
writer.add_graph(model, X)
writer.flush()

In [10]:

tracker = EmissionsTracker(project_name="MLP")
tracker.start()

EPOCHS = 400
model = SuicideRegressorDrop(in_features=9)
criterion = nn.MSELoss()
optim = torch.optim.Adam(
    params=model.parameters(),
    lr=5e-3,
    weight_decay=1e-4
)




for epoch in range(EPOCHS):
    model.train()
    loss = 0
    for batch_ftrs, batch_labels in train_loader:
        optim.zero_grad()
        y_pred = model(batch_ftrs)
        train_loss = criterion(y_pred[:,0], batch_labels)
        train_loss.backward()

        optim.step()
        loss += train_loss.item()

    loss = loss / len(train_loader)

    # compute validation loss
    val_loss = 0
    with torch.no_grad():
        model.eval()
        for batch_ftrs, batch_labels in val_loader:
            y_pred = model(batch_ftrs)
            cur_loss = criterion(y_pred[:,0], batch_labels)
            val_loss += cur_loss.item()
        val_loss = val_loss / len(val_loader)
    # prints
    if epoch % 200 == 0:
        print("epoch : {}/{}, train_loss = {:.6f}".format(epoch + 1, EPOCHS, loss))
        print("epoch : {}/{}, val_loss = {:.6f}".format(epoch + 1, EPOCHS, val_loss))
     # add scalars to tensorboard
    writer.add_scalar(f"{criterion}/train", loss, epoch + 1)
    writer.add_scalar(f"{criterion}/val", val_loss, epoch + 1)
    writer.add_scalars("comp", {'train':loss, 'val':val_loss}, epoch+1)

    writer.flush()

tracker.stop()

[codecarbon INFO @ 10:23:04] [setup] RAM Tracking...
[codecarbon INFO @ 10:23:04] [setup] GPU Tracking...
[codecarbon INFO @ 10:23:04] No GPU found.
[codecarbon INFO @ 10:23:04] [setup] CPU Tracking...
[codecarbon WARNING @ 10:23:04] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 10:23:06] CPU Model on constant consumption mode: Apple M1 Pro
[codecarbon INFO @ 10:23:06] >>> Tracker's metadata:
[codecarbon INFO @ 10:23:06]   Platform system: macOS-10.16-x86_64-i386-64bit
[codecarbon INFO @ 10:23:06]   Python version: 3.10.9
[codecarbon INFO @ 10:23:06]   Available RAM : 16.000 GB
[codecarbon INFO @ 10:23:06]   CPU count: 8
[codecarbon INFO @ 10:23:06]   CPU model: Apple M1 Pro
[codecarbon INFO @ 10:23:06]   GPU count: None
[codecarbon INFO @ 10:23:06]   GPU model: None


epoch : 1/400, train_loss = 479.082031
epoch : 1/400, val_loss = 541.569336


In [24]:
# torch.save(model, "torch_models/bn_do.pickle")

In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

with torch.no_grad():
    model.eval()
    pred = model(torch.tensor(X_val).float())
    custom_mlp_pred = pred[:, 0]


## compare with sklearn MLP

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_model = MLPRegressor(
    hidden_layer_sizes=[512,256,64,8],
    max_iter=3000,
    activation='relu')
mlp_model.fit(X_train_std,Y_train)

In [ ]:
mlp_pred = mlp_model.predict(X_test_std)
metrics = [mean_squared_error, mean_absolute_error, median_absolute_error]
model_preds = {
    'mlp': mlp_pred,
    'custom_mlp': custom_mlp_pred
}


for metric in metrics:
    print(f"{metric.__name__} eval:")
    for model, pred in model_preds.items():
        score = metric(y_true=Y_test, y_pred=pred)
        print(f"{model}: {score}")
    print('-'*10)

In [ ]:
n_samples = 50
pred_df = pd.DataFrame({
    'sklearn_mlp_pred': mlp_model.predict(X_test_std)[:n_samples],
    'custom_mlp_pred' : custom_mlp_pred[:n_samples],
    'ground_truth': Y_test[:n_samples]
})
sns.scatterplot(pred_df, markers=True, alpha=.6)
sns.lineplot(pred_df)